A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

ROIC utilizada por Greenblatt é diferente da fórmula tradicional

ROIC = EBIT / (Capital Circulante Líquido + Ativos Fixos Líquidos)

Onde o Capital circulante Líquido é um proxy do Capital de Giro da Empresa (Ativo Circulante – Passivo Circulante), os Ativos Fixos Líquidos é proxy dos ativos permanentes (imobilizado e intangíveis, utilizados para gerar retorno no longo prazo. A Taxa de Retorno de Lucros para uma empresa é a fórmula tradicional: Taxa de Retorno = Lucro Operacional / Valor da Empresa

Estes dois indicadores são colocados cada um em um ranking, da melhor (rank 1) para a pior empresa. Depois somamos os números das duas colunas, quanto menor a soma, melhor.

Para os princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e  seguradoras devem ser excluídas da seleção. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
from IPython.display import display, HTML


def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))


def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None

def setor(setor:int):

    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])



dados do site https://www.fundamentus.com.br/resultado.php

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
data_funds = pd.DataFrame(funda)
data_funds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [3]:
earning_yield = 'EV/EBIT'  #Quanto menor melhor
return_on_capital = 'ROIC' #ROIC – Quanto maior melhor

### filtros e ranking

- Empresa precisa ter ROIC positivo
- Empresa precisa ter Lucro Operacional positivo.
- Volume de negociação diário medio maior que R$ 1000000.000,00 nos últimos 2 meses.

ROIC – Quanto maior melhor

EV/EBIT – Quanto menor melhor

In [4]:
funds = data_funds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]
funds =  funds[funds['Liq.2meses'] > 1000000]     #Volume diário médio negociado nos últimos 2 meses

""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

### lista ranqueada

In [5]:
funds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a',
       'Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final'],
      dtype='object')

In [6]:
funds[['Papel','EV/EBITDA', 'ROIC', 'Liq.2meses','Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final']].head(20)

,Papel,EV/EBITDA,ROIC,Liq.2meses,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,0.59,75.32,7.580140e+07,1.0,1.0,2.0
2,PLPL3,3.00,45.40,1.154030e+07,6.0,2.0,8.0
3,WIZC3,1.36,31.27,2.509090e+06,2.0,7.0,9.0
4,PETR4,2.60,20.22,1.594220e+09,8.0,20.0,28.0
5,PETR3,2.70,20.22,4.953660e+08,9.0,20.0,29.0
6,KEPL3,4.06,26.04,1.585650e+07,19.0,11.0,30.0
7,SYNE3,2.20,17.95,2.402460e+06,5.0,28.0,33.0
8,LREN3,2.78,19.09,2.501670e+08,10.0,23.0,33.0
9,ASAI3,3.25,18.89,1.149950e+08,13.0,24.0,37.0
10,POMO3,4.48,20.99,3.093080e+06,21.0,18.0,39.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [7]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


RPAD5 RPAD3 RPAD6 BGIP3 BGIP4 PINE4 PINE3 BMEB3 BNBR3 BBAS3 BAZA3 BMEB4 BRSR6 ABCB4 BMGB4 BRSR3 BBDC3 BEES3 BEES4 BRSR5 BBDC4 SANB3 ITSA4 ITSA3 SANB11 BPAC5 BRBI11 SANB4 ITUB3 ITUB4 BMIN4 BPAC11 BMIN3 BPAN4 MERC4 BSLI3 BPAC3 BSLI4
IRBR3 WIZC3 BBSE3 PSSA3 CXSE3
AMBP3 SAPR4 SAPR11 SAPR3 CSMG3 SBSP3 ORVR3 RNEW11 RNEW3 RNEW4 CEED3 GEPA3 GEPA4 LIGT3 CLSC3 COCE3 CMIG4 COCE5 ISAE4 CLSC4 ENGI4 CBEE3 CEEB5 ENGI11 CEEB3 CMIG3 EQMA3B ISAE3 EQPA3 NEOE3 TAEE3 TAEE11 TAEE4 ENGI3 EGIE3 CPFE3 CEBR3 CEBR5 EQPA5 ALUP4 ALUP11 ALUP3 CEBR6 EQPA7 REDE3 ELET3 ELET6 EKTR4 CPLE3 CPLE5 CPLE6 AFLT3 EQTL3 ENMT4 ENMT3 LIPR3 SRNA3 EMAE4 AURE3 GPAR3 ENEV3


#### sem bancos

In [8]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(bancos)],10
)
          

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,41.60,10.17,1.91,0.763,3.80,0.561,7.88,0.93,-3.60,0.60,0.59,81.86,7.63,1.15,75.32,18.82,75801400.0,1.405080e+10,0.00,18.52,1.0,1.0,2.0,PSSA
PLPL3,11.66,6.91,2.81,0.918,8.63,0.921,1.49,3.38,4.52,3.08,3.00,27.13,15.08,3.69,45.40,40.58,11540300.0,8.472350e+08,0.69,29.69,6.0,2.0,8.0,PLPL
WIZC3,5.91,5.90,1.73,0.752,4.17,0.386,-3.91,1.50,-1.17,1.62,1.36,50.28,23.61,0.66,31.27,29.30,2509090.0,5.463700e+08,0.63,12.90,2.0,7.0,9.0,WIZC
PETR4,30.57,10.76,1.08,0.803,28.41,0.350,-6.61,1.87,-0.63,3.43,2.60,42.87,7.54,0.69,20.22,10.00,1594220000.0,3.660060e+11,1.02,13.92,8.0,20.0,28.0,PETR
KEPL3,7.72,6.97,1.87,0.863,10.68,0.870,2.62,4.97,6.40,4.64,4.06,17.37,12.39,1.98,26.04,26.89,15856500.0,7.407810e+08,0.44,21.60,19.0,11.0,30.0,KEPL
SYNE3,5.50,1.54,0.78,0.618,119.11,0.267,0.90,1.72,-5.30,2.56,2.20,36.05,48.61,5.21,17.95,50.87,2402460.0,1.073780e+09,0.76,3.68,5.0,28.0,33.0,SYNE
LREN3,12.91,11.43,1.27,0.948,5.06,0.672,2.93,4.54,5.02,3.93,2.78,20.87,8.29,1.61,19.09,11.11,250167000.0,1.077300e+10,0.09,16.83,10.0,23.0,33.0,LREN
ASAI3,8.70,15.30,2.24,0.159,1.07,0.258,86.50,2.20,-0.49,4.24,3.25,7.24,1.04,1.01,18.89,14.63,114995000.0,5.255000e+09,3.15,20.87,13.0,24.0,37.0,ASAI
POMO3,5.16,4.89,1.46,0.682,11.43,0.621,2.27,4.19,19.49,5.02,4.48,16.28,14.22,1.84,20.99,29.80,3093080.0,4.026610e+09,0.81,27.04,21.0,18.0,39.0,POMO
QUAL3,2.18,94.11,0.48,0.392,0.00,0.142,4.01,1.21,-0.44,3.14,1.76,32.39,0.95,1.10,17.15,0.51,4689230.0,1.282270e+09,1.47,-6.55,7.0,33.0,40.0,QUAL


#### sem seguradora

In [9]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(seguros)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,11.66,6.91,2.81,0.918,8.63,0.921,1.49,3.38,4.52,3.08,3.00,27.13,15.08,3.69,45.40,40.58,11540300.0,8.472350e+08,0.69,29.69,6.0,2.0,8.0,PLPL
PETR4,30.57,10.76,1.08,0.803,28.41,0.350,-6.61,1.87,-0.63,3.43,2.60,42.87,7.54,0.69,20.22,10.00,1594220000.0,3.660060e+11,1.02,13.92,8.0,20.0,28.0,PETR
KEPL3,7.72,6.97,1.87,0.863,10.68,0.870,2.62,4.97,6.40,4.64,4.06,17.37,12.39,1.98,26.04,26.89,15856500.0,7.407810e+08,0.44,21.60,19.0,11.0,30.0,KEPL
SYNE3,5.50,1.54,0.78,0.618,119.11,0.267,0.90,1.72,-5.30,2.56,2.20,36.05,48.61,5.21,17.95,50.87,2402460.0,1.073780e+09,0.76,3.68,5.0,28.0,33.0,SYNE
LREN3,12.91,11.43,1.27,0.948,5.06,0.672,2.93,4.54,5.02,3.93,2.78,20.87,8.29,1.61,19.09,11.11,250167000.0,1.077300e+10,0.09,16.83,10.0,23.0,33.0,LREN
ASAI3,8.70,15.30,2.24,0.159,1.07,0.258,86.50,2.20,-0.49,4.24,3.25,7.24,1.04,1.01,18.89,14.63,114995000.0,5.255000e+09,3.15,20.87,13.0,24.0,37.0,ASAI
POMO3,5.16,4.89,1.46,0.682,11.43,0.621,2.27,4.19,19.49,5.02,4.48,16.28,14.22,1.84,20.99,29.80,3093080.0,4.026610e+09,0.81,27.04,21.0,18.0,39.0,POMO
QUAL3,2.18,94.11,0.48,0.392,0.00,0.142,4.01,1.21,-0.44,3.14,1.76,32.39,0.95,1.10,17.15,0.51,4689230.0,1.282270e+09,1.47,-6.55,7.0,33.0,40.0,QUAL
VLID3,24.56,5.28,1.12,0.926,12.39,0.683,1.98,5.13,4.67,5.09,4.01,18.04,17.48,2.88,18.20,21.22,11576900.0,1.794710e+09,0.33,2.55,23.0,26.0,49.0,VLID
LEVE3,30.42,7.62,5.37,0.905,7.77,1.147,6.91,5.06,-6.30,5.93,5.25,17.89,11.87,1.38,36.21,70.45,10160200.0,7.684440e+08,1.39,15.76,46.0,4.0,50.0,LEVE


#### sem utilidade publica

In [10]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(util)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,41.60,10.17,1.91,0.763,3.80,0.561,7.88,0.93,-3.60,0.60,0.59,81.86,7.63,1.15,75.32,18.82,75801400.0,1.405080e+10,0.00,18.52,1.0,1.0,2.0,PSSA
PLPL3,11.66,6.91,2.81,0.918,8.63,0.921,1.49,3.38,4.52,3.08,3.00,27.13,15.08,3.69,45.40,40.58,11540300.0,8.472350e+08,0.69,29.69,6.0,2.0,8.0,PLPL
WIZC3,5.91,5.90,1.73,0.752,4.17,0.386,-3.91,1.50,-1.17,1.62,1.36,50.28,23.61,0.66,31.27,29.30,2509090.0,5.463700e+08,0.63,12.90,2.0,7.0,9.0,WIZC
PETR4,30.57,10.76,1.08,0.803,28.41,0.350,-6.61,1.87,-0.63,3.43,2.60,42.87,7.54,0.69,20.22,10.00,1594220000.0,3.660060e+11,1.02,13.92,8.0,20.0,28.0,PETR
KEPL3,7.72,6.97,1.87,0.863,10.68,0.870,2.62,4.97,6.40,4.64,4.06,17.37,12.39,1.98,26.04,26.89,15856500.0,7.407810e+08,0.44,21.60,19.0,11.0,30.0,KEPL
SYNE3,5.50,1.54,0.78,0.618,119.11,0.267,0.90,1.72,-5.30,2.56,2.20,36.05,48.61,5.21,17.95,50.87,2402460.0,1.073780e+09,0.76,3.68,5.0,28.0,33.0,SYNE
LREN3,12.91,11.43,1.27,0.948,5.06,0.672,2.93,4.54,5.02,3.93,2.78,20.87,8.29,1.61,19.09,11.11,250167000.0,1.077300e+10,0.09,16.83,10.0,23.0,33.0,LREN
ASAI3,8.70,15.30,2.24,0.159,1.07,0.258,86.50,2.20,-0.49,4.24,3.25,7.24,1.04,1.01,18.89,14.63,114995000.0,5.255000e+09,3.15,20.87,13.0,24.0,37.0,ASAI
POMO3,5.16,4.89,1.46,0.682,11.43,0.621,2.27,4.19,19.49,5.02,4.48,16.28,14.22,1.84,20.99,29.80,3093080.0,4.026610e+09,0.81,27.04,21.0,18.0,39.0,POMO
QUAL3,2.18,94.11,0.48,0.392,0.00,0.142,4.01,1.21,-0.44,3.14,1.76,32.39,0.95,1.10,17.15,0.51,4689230.0,1.282270e+09,1.47,-6.55,7.0,33.0,40.0,QUAL


#### sem as 3 classes

In [11]:
b_print(
    funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,11.66,6.91,2.81,0.918,8.63,0.921,1.49,3.38,4.52,3.08,3.00,27.13,15.08,3.69,45.40,40.58,11540300.0,8.472350e+08,0.69,29.69,6.0,2.0,8.0,PLPL
PETR4,30.57,10.76,1.08,0.803,28.41,0.350,-6.61,1.87,-0.63,3.43,2.60,42.87,7.54,0.69,20.22,10.00,1594220000.0,3.660060e+11,1.02,13.92,8.0,20.0,28.0,PETR
KEPL3,7.72,6.97,1.87,0.863,10.68,0.870,2.62,4.97,6.40,4.64,4.06,17.37,12.39,1.98,26.04,26.89,15856500.0,7.407810e+08,0.44,21.60,19.0,11.0,30.0,KEPL
SYNE3,5.50,1.54,0.78,0.618,119.11,0.267,0.90,1.72,-5.30,2.56,2.20,36.05,48.61,5.21,17.95,50.87,2402460.0,1.073780e+09,0.76,3.68,5.0,28.0,33.0,SYNE
LREN3,12.91,11.43,1.27,0.948,5.06,0.672,2.93,4.54,5.02,3.93,2.78,20.87,8.29,1.61,19.09,11.11,250167000.0,1.077300e+10,0.09,16.83,10.0,23.0,33.0,LREN
ASAI3,8.70,15.30,2.24,0.159,1.07,0.258,86.50,2.20,-0.49,4.24,3.25,7.24,1.04,1.01,18.89,14.63,114995000.0,5.255000e+09,3.15,20.87,13.0,24.0,37.0,ASAI
POMO3,5.16,4.89,1.46,0.682,11.43,0.621,2.27,4.19,19.49,5.02,4.48,16.28,14.22,1.84,20.99,29.80,3093080.0,4.026610e+09,0.81,27.04,21.0,18.0,39.0,POMO
QUAL3,2.18,94.11,0.48,0.392,0.00,0.142,4.01,1.21,-0.44,3.14,1.76,32.39,0.95,1.10,17.15,0.51,4689230.0,1.282270e+09,1.47,-6.55,7.0,33.0,40.0,QUAL
VLID3,24.56,5.28,1.12,0.926,12.39,0.683,1.98,5.13,4.67,5.09,4.01,18.04,17.48,2.88,18.20,21.22,11576900.0,1.794710e+09,0.33,2.55,23.0,26.0,49.0,VLID
LEVE3,30.42,7.62,5.37,0.905,7.77,1.147,6.91,5.06,-6.30,5.93,5.25,17.89,11.87,1.38,36.21,70.45,10160200.0,7.684440e+08,1.39,15.76,46.0,4.0,50.0,LEVE


outros filtros

In [12]:
funds2 =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds2 =  funds2[(funds['P/L'] < 30)]           # não estar excessivamente cara
funds2 =  funds2[funds['Dív.Brut/ Patrim.'] < 3.5]  # endividamento sob controle
funds2 =  funds2[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos

In [13]:
b_print(
    funds2[(~funds2['Papel'].isin(bancos))&(~funds2['Papel'].isin(seguros))&(~funds2['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,11.66,6.91,2.81,0.918,8.63,0.921,1.49,3.38,4.52,3.08,3.00,27.13,15.08,3.69,45.40,40.58,11540300.0,8.472350e+08,0.69,29.69,6.0,2.0,8.0,PLPL
PETR4,30.57,10.76,1.08,0.803,28.41,0.350,-6.61,1.87,-0.63,3.43,2.60,42.87,7.54,0.69,20.22,10.00,1594220000.0,3.660060e+11,1.02,13.92,8.0,20.0,28.0,PETR
KEPL3,7.72,6.97,1.87,0.863,10.68,0.870,2.62,4.97,6.40,4.64,4.06,17.37,12.39,1.98,26.04,26.89,15856500.0,7.407810e+08,0.44,21.60,19.0,11.0,30.0,KEPL
SYNE3,5.50,1.54,0.78,0.618,119.11,0.267,0.90,1.72,-5.30,2.56,2.20,36.05,48.61,5.21,17.95,50.87,2402460.0,1.073780e+09,0.76,3.68,5.0,28.0,33.0,SYNE
LREN3,12.91,11.43,1.27,0.948,5.06,0.672,2.93,4.54,5.02,3.93,2.78,20.87,8.29,1.61,19.09,11.11,250167000.0,1.077300e+10,0.09,16.83,10.0,23.0,33.0,LREN
ASAI3,8.70,15.30,2.24,0.159,1.07,0.258,86.50,2.20,-0.49,4.24,3.25,7.24,1.04,1.01,18.89,14.63,114995000.0,5.255000e+09,3.15,20.87,13.0,24.0,37.0,ASAI
POMO3,5.16,4.89,1.46,0.682,11.43,0.621,2.27,4.19,19.49,5.02,4.48,16.28,14.22,1.84,20.99,29.80,3093080.0,4.026610e+09,0.81,27.04,21.0,18.0,39.0,POMO
VLID3,24.56,5.28,1.12,0.926,12.39,0.683,1.98,5.13,4.67,5.09,4.01,18.04,17.48,2.88,18.20,21.22,11576900.0,1.794710e+09,0.33,2.55,23.0,26.0,49.0,VLID
LEVE3,30.42,7.62,5.37,0.905,7.77,1.147,6.91,5.06,-6.30,5.93,5.25,17.89,11.87,1.38,36.21,70.45,10160200.0,7.684440e+08,1.39,15.76,46.0,4.0,50.0,LEVE
CEAB3,12.92,8.80,1.20,0.522,2.67,0.396,3.48,4.26,-2.27,4.18,3.05,12.23,5.93,1.30,14.97,13.68,43603600.0,3.308480e+09,0.45,16.37,12.0,43.0,55.0,CEAB


retirar BDR

In [14]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds[funds['Papel'].str.contains(padrao)]


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
27,AURA33,36.8,-53.65,5.79,2.481,2.15,1.195,9.35,6.7,-2.76,...,1.55,27.04,-10.79,11333300.0,1.380600e+09,1.99,13.82,78.0,10.0,88.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.

In [15]:
data_funds.loc['INEP4']

Cotação              1.540000e+00
P/L                 -2.900000e-01
P/VP                -5.000000e-02
PSR                  1.550400e+01
Div.Yield            0.000000e+00
P/Ativo              7.300000e-02
P/Cap.Giro          -6.000000e-02
P/EBIT              -9.400000e-01
P/Ativ Circ.Liq     -3.000000e-02
EV/EBIT             -1.373000e+01
EV/EBITDA           -1.510000e+01
Mrg Ebit            -1.647020e+03
Mrg. Líq.           -5.378760e+03
Liq. Corr.           8.000000e-02
ROIC                -8.310000e+00
ROE                  1.606000e+01
Liq.2meses           4.717080e+04
Patrim. Líq         -1.448530e+09
Dív.Brut/ Patrim.   -6.400000e-01
Cresc. Rec.5a       -2.164000e+01
Name: INEP4, dtype: float64